In [1]:
#load libraries 
import time,os
from matplotlib.offsetbox import AnchoredText
import sys,pandas as pd, matplotlib , matplotlib.pyplot as plt, matplotlib.lines, numpy as np, math, pylab
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#import cudf
#import ROOT
#from ROOT import TFile

#import root_pandas

#from toggle_cell import hide_cell 
#import uproot3
%matplotlib inline

#presets for plot style
from scipy.constants import golden
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (7, 7/golden),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

viridis = cm.get_cmap('viridis', 12)
inferno = cm.get_cmap('inferno', 12)
autumn = cm.get_cmap('autumn', 12)

#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
#tf.config.experimental.set_memory_growth(physical_devices[1], True)
rc = plt.rc
rc('text', usetex=False)
import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=False)
rc('font', size=15)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

plt.rcParams['figure.facecolor']='white'
plt.rcParams['savefig.facecolor']='white'
plt.rcParams['savefig.bbox']='tight'

import root_pandas,glob

import gc

import uproot
def read_n(pattern, n):
    if not '*' in pattern:
        a = root_pandas.read_root(pattern,chunksize=int(n))
    else:
        a = root_pandas.read_root(glob.glob(pattern),chunksize=int(n))
    for b in a:
        del a
        print(n)
        return b

from scipy.optimize import curve_fit

image_dir='images'

#import misc_util#correlation_plots

import R2h_module
#import imp
#imp.reload(R2h_module)

import mplhep as hep
hep.style.use("CMS")

In [2]:
dfs_pair = {}
dfs_trigger = {}

In [3]:
#get GiBUU
import uproot

def getdatainChunks(filename,treename):
    dataframe =pd.DataFrame()
    file = uproot.open(filename) 
    tree=file[treename]
    df=tree.arrays(filter_name='*',library='pd')
    return df

pathGIBUU="./"
for target in ['D','C','Fe','Pb']:
    for suffix in "_tweak_harut",:
        #GiBUU
        print (target+suffix)
        print (' trigger')
        dfs_trigger[target]= getdatainChunks('%sGiBUU_Pairs_%s.root'%(pathGIBUU,target+suffix), 
                                                               '%s_trigger'%target)
        print (' pairs')
        dfs_pair[target]= getdatainChunks('%sGiBUU_Pairs_%s.root'%(pathGIBUU,target+suffix),target)
print (dfs_trigger.keys())

D_tweak_harut
 trigger
 pairs
C_tweak_harut
 trigger
 pairs
Fe_tweak_harut
 trigger
 pairs
Pb_tweak_harut
 trigger
 pairs
dict_keys(['D', 'C', 'Fe', 'Pb'])


In [4]:
print(len(dfs_trigger['D'].query("h1_pid==211 and h1_th>10 ")))

0


In [5]:
# additional cuts that were not applied to the GiBUU set in PRL,
# but we should see what happens if we include them.  
pThetaCuts_trig=' ((h1_pid==211 and h1_th>10) or (h1_pid==2212))'

#pThetaCuts_trig+=" and h1_th>20 and h1_p<2.7"

pThetaCuts_pair=pThetaCuts_trig 
pThetaCuts_pair += " and " + pThetaCuts_trig.replace("h1_", "h2_")

In [6]:
%%time
import R2h_module as r2h

for key in dfs_pair.keys():
    
    isMC=True
    print ('\nkey: ',key)
    dfs_pair[key].rename(columns={'h1_FidCut':'h1_FidCutPiPlus'}, inplace=True)
    dfs_pair[key].rename(columns={'h2_FidCut':'h2_FidCutPiPlus'}, inplace=True)
    dfs_pair[key] = r2h.applyCuts(dfs_pair[key],isMC=isMC, nomCuts=True)
    dfs_pair[key] = r2h.applyCutsPair(dfs_pair[key],isMC=isMC, nomCuts=True, h2Proton=True)
for key in dfs_trigger.keys():
    isMC = False
    if 'GiBUU' in key:
        isMC=True    
    print (key, isMC)
    dfs_trigger[key].rename(columns={'h1_FidCut':'h1_FidCutPiPlus'}, inplace=True)
    dfs_trigger[key] = r2h.applyCuts(dfs_trigger[key],isTrigger=True, isMC=isMC, nomCuts=True)
    print (' ')
    


key:  D
Entries before cut  2348393
Q2>1.0 and Q2<4.0 2347490  (99.96  %)
0.2<h1_p<5.0  2347490  (100.00  %)
inelasticity < 0.85 2347490  (100.00  %)
h1_pid = pions (trigger) 2347490  (100.00  %)
2.2 < nu <4.2 2254015  (96.02  %)
Starting election on dipion variables
This is MC
1.0< Q2 <4.0 2254015  (100.00  %)
2.2 < nu < 4.2 2254015  (100.00  %)
secondary hadrons are protons 925561  (41.06  %)
leading hadrons are pions 925561  (100.00  %)
0.2<h2_p<5.0 925561  (100.00  %)
0.2<h1_p<5.0 925561  (100.00  %)

key:  C
Entries before cut  2074556
Q2>1.0 and Q2<4.0 2073705  (99.96  %)
0.2<h1_p<5.0  2073705  (100.00  %)
inelasticity < 0.85 2073705  (100.00  %)
h1_pid = pions (trigger) 2073705  (100.00  %)
2.2 < nu <4.2 1989368  (95.93  %)
Starting election on dipion variables
This is MC
1.0< Q2 <4.0 1989368  (100.00  %)
2.2 < nu < 4.2 1989368  (100.00  %)
secondary hadrons are protons 880844  (44.28  %)
leading hadrons are pions 880844  (100.00  %)
0.2<h2_p<5.0 880844  (100.00  %)
0.2<h1_p<5.

UndefinedVariableError: name 'h1_deltaZ' is not defined

In [7]:
def get_values(slicevar='drap', slicevartex='\Delta Y', slicebins=(-1,0,1,2,3), saveFig=False,
              pThetaCutsGiBUU=True, pMin=.4, pT1min=.25, pT2min=.25,
               suptitle=None,nbins_data=8, nbins_GiBUU=16, GiBUU_suffix="",
              addCuts=None, addCutsTrig=None, sliceNh=False):
    #unbinned.
    if slicevar is None:
        slicebins=[0,1]
    
    data=[]
    GiBUU=[]
    sliceinfo={}
    
    #guess for the systematic errors from sources not inves
    systPct=0
    pid=211
    
    integral_data_D = 0
    for k in range(len(slicebins)-1):
        data.append({})
        GiBUU.append({})
        slice_range=slicebins[k:k+2]

        data_D=None
        stat_D=None
        Alist = enumerate("D C Fe Pb".split())
        for i, A in Alist:

            

            q = f"h1_pid==211   and h1_cm_pt > {pT1min} and "+\
                 f"h2_cm_pt > {pT2min} and h1_z>0.5 and h2_pid==2212 and h2_p>{pMin}"
            
            if slicevar != None:
                q+= f" and {slicevar}>={slice_range[0]} and {slicevar}<{slice_range[1]}"
            
            if addCuts is not None:
                q+= f" and {addCuts}"

            which_set=A

            bins = np.linspace(0,np.pi,nbins_data+1)

            N2, x = np.histogram(dfs_pair[A].query(q)['dphi'], bins=bins)
            print("N2", N2)
            
            qtrig = f"h1_pid==211 and h1_th*{180/np.pi}>10 and  h1_cm_pt > {pT1min}"
            
            #if sliceNh:
            #    qtrig+= f" and {slicevar}>={slice_range[0]} and {slicevar}<{slice_range[1]}"
            
            #if addCutsTrig is not None:
            #    qtrig += f" and {addCutsTrig}"
            
            N1 = len(dfs_trigger[A].query(qtrig))
            print("N1", N1)
            x = (x[1:]+x[:-1])/2
            dx = x[1]-x[0]

            S = N2/dx/N1/2
            dSstat = np.sqrt(N2*(N1-N2)/N1**3)/dx/2

            C=S
            dCstat = dSstat
            


            if A=="D":  
                integral_data_D+=sum(C)*dx*2*(slice_range[1]-slice_range[0])

            dCsyst = C*systPct/100

            if A=='D' or "D_" in A:
                data_D=C
                stat_D=dCstat

            R_data = C/data_D
            dR_data_stat = C/data_D*np.hypot(dCstat/C, stat_D/data_D)
            dCtot = np.hypot(dCstat, dCsyst)

            
            data[k][A] = pd.DataFrame(dict(C=C, dCstat=dCstat, dCsyst=dCsyst, 
                                            phi=x,phi_low=bins[:-1], phi_up=bins[1:],
                                                S=S, dSstat=dSstat,
                                                R=R_data,
                                                dRstat=dR_data_stat, dRsyst=R_data*0.0,
                                        N2h=N2, N1=[N1]*len(N2)))
            data[k][A+"_N1"] = N1
            if "D_" in A:
                continue
        
        
        if slicevar is not None:
            sliceinfo[k] = f"{slice_range[0]:.1f} < ${slicevartex}$< {slice_range[1]:.1f}"
            
        else :
            sliceinfo[k] = f""
    #for A in "D C Fe Pb".split():
    #    for k in range(len(slicebins)-1):
    #        data[k][A]['C']/=integral_data_D
    #        data[k][A]['dCstat']/=integral_data_D
    #        data[k][A]['dCsyst']/=integral_data_D
            
    return data, GiBUU, sliceinfo

In [8]:
GiBUU_integrated, _, _ = get_values(None,  pThetaCutsGiBUU=True,GiBUU_suffix=None, pMin=0.35,pT1min=0.07, pT2min=0.07)


N2 [  7625   8544  10463  14774  22616  37970  67669 106564]
N1 905146
N2 [ 11727  12870  15660  20357  29452  44973  70370 100332]
N1 749730
N2 [ 6986  7650  9085 11968 16009 23129 33540 45197]
N1 306985
N2 [ 8790  9302 11076 14043 19094 26196 37432 48194]
N1 393290


In [9]:
GiBUU_dY, _, _ = get_values("drap-ycm", "\Delta Y^*", np.linspace(-1.0, 2.0, 7),GiBUU_suffix=None, pMin=0.35,pT1min=0.07,pT2min=0.07)


N2 [   58    89   120   208   422  1100  4551 13935]
N1 905146
N2 [  187   241   317   469   841  1772  4774 11127]
N1 749730
N2 [ 141  147  218  289  521  931 2108 4378]
N1 306985
N2 [ 177  200  270  366  606 1066 2284 4280]
N1 393290
N2 [  570   634   862  1375  2490  5705 15078 28144]
N1 905146
N2 [ 1319  1450  1799  2539  4040  7551 14991 24405]
N1 749730
N2 [  847   963  1106  1609  2382  3898  6844 10426]
N1 306985
N2 [ 1135  1157  1568  1936  2892  4458  7436 11078]
N1 393290
N2 [ 1668  1905  2429  3758  6316 12428 22762 32643]
N1 905146
N2 [ 3011  3203  4158  5778  8911 14500 22972 31468]
N1 749730
N2 [ 1857  2067  2564  3502  4923  7455 10900 14076]
N1 306985
N2 [ 2397  2553  3123  4021  5909  8572 12287 15161]
N1 393290
N2 [ 2655  2948  3664  5253  7975 12110 17023 20499]
N1 905146
N2 [ 3623  4143  4981  6342  9220 13167 17765 21145]
N1 749730
N2 [ 2055  2321  2734  3612  4831  6561  8663 10162]
N1 306985
N2 [ 2618  2776  3306  4224  5745  7374  9586 11081]
N1 393290
N2 [2091

In [10]:
import pickle as pkl

In [11]:
import pickle
with open("pion_proton_GiBUU.pkl", "wb") as f:
    pickle.dump({"all": GiBUU_integrated, "dY":GiBUU_dY}, f, pickle.HIGHEST_PROTOCOL)